In [1]:
import pandas as pd
import csv
import subprocess
import glob
from pathlib import Path
import seaborn as sns

In [2]:
# MDB to CSV
def show_data(path='2562.mdb', table='EventCustom'):
    tables = subprocess.check_output(["mdb-export", path, table])
    return tables.decode().split('\n')
 
def convert_EventCustom(path, table):
    d = show_data(path, table)
    columns = d[0].split(',')
    data = [i.split(',') for i in d[1:]]
    df = pd.DataFrame(columns=columns, data=data)
    df = df.dropna()
    df.to_csv('EventCustom'+path[:4] + '.csv', encoding='utf-8', quoting=csv.QUOTE_NONE)
    return df

def convert_Event(path, table):
    d = show_data(path, table)
    columns = d[0].split(',')
    data = [i.split(',') for i in d[1:]]
    df = pd.DataFrame(columns=columns, data=data)
    df = df.dropna()
    df.to_csv('Event'+path[:4] + '.csv', encoding='utf-8', quoting=csv.QUOTE_NONE)
    return df

def convert_EventCause(path, table):
    d = show_data(path, table)
    columns = d[0].split(',')
    data = [i.split(',') for i in d[1:]]
    df = pd.DataFrame(data=data)
    df = df.iloc[:, 0:17]
    df.columns = columns
    df = df.dropna()    
    df.to_csv('EventCause'+path[:4] + '.csv', encoding='utf-8')
    return df
    ECustom = convert_EventCustom(path.name,'EventCustom')

In [3]:
ls

2556.mdb                       EventCause2559.csv
2557.mdb                       EventCause2560.csv
2558.mdb                       EventCause2561.csv
2559.mdb                       EventCause2562.csv
2560.mdb                       EventCustom2556.csv
2561.mdb                       EventCustom2557.csv
2562.mdb*                      EventCustom2558.csv
7Years_PEA.ipynb               EventCustom2559.csv
ALLPEA_7Y.sav                  EventCustom2560.csv
All_Ratio.csv                  EventCustom2561.csv
Data_Pipeline.ipynb            EventCustom2562.csv
Data_Pipeline_Dashboard.ipynb  EventForML.csv
EquipCode.csv                  Merge_Event.csv
Event2556.csv                  Merge_Event.csv 2.zip
Event2557.csv                  Merge_Event.csv.zip
Event2558.csv                  Untitled.ipynb
Event2559.csv                  combined_EventCause_csv.csv
Event2560.csv                  combined_EventCustom_csv.csv
Event2561.csv                  combined_Event_csv.csv
Event2562.csv              

In [ ]:
#Define all file to glob
for path in Path('./').rglob('*.mdb'):
    Event = convert_Event(path.name,'Event')
    ECustom = convert_EventCustom(path.name,'EventCustom')
    ECause = convert_EventCause(path.name,'EventCause')

In [5]:
# Glob all file
extension = 'csv'
all_Event = [i for i in glob.glob('*Event25*.{}'.format(extension))]
all_EventCustom = [i for i in glob.glob('*EventCustom*.{}'.format(extension))]
all_EventCause = [i for i in glob.glob('*EventCause*.{}'.format(extension))]

In [6]:
#Combine all files in the list
combined_Event_csv = pd.concat([pd.read_csv(f) for f in all_Event],ignore_index=True,sort=False)
combined_EventCustom_csv = pd.concat([pd.read_csv(f) for f in all_EventCustom],ignore_index=True,sort=False)
combined_EventCause_csv = pd.concat([pd.read_csv(f) for f in all_EventCause],ignore_index=True,sort=False)
combined_EventCause_csv = combined_EventCause_csv.replace({'"': ''}, regex=True)

/Users/macbookpro/opt/anaconda3/envs/ML/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/macbookpro/opt/anaconda3/envs/ML/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/macbookpro/opt/anaconda3/envs/ML/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
# #Dump file to csv
# combined_Event_csv.to_csv('combined_Event_csv.csv',encoding='utf-8', quoting=csv.QUOTE_NONE,index = False)
# combined_EventCustom_csv.to_csv('combined_EventCustom_csv.csv',encoding='utf-8', quoting=csv.QUOTE_NONE,index = False)
# # combined_EventCause_csv.to_csv('combined_EventCause_csv.csv',encoding='utf-8', quoting=csv.QUOTE_NONE,index = False)

In [7]:
#Cleaning EventCustom
ECus = combined_EventCustom_csv
ECus = ECus.drop('Unnamed: 0', axis=1)
ECus['All_Cus'] = ECus['Cus_A1']+ECus['Cus_A2']+ECus['Cus_A3']+ECus['Cus_A4']+ECus['Cus_A5']
ECus = ECus[['EventID','DateOutage','TimeOutage','KnowCause','CauseType','TotalTime','FeederCode','PeaCode','CausePea','All_Cus']]
ECus = ECus.loc[ECus['TotalTime'] != 0]
ECus = ECus.loc[ECus['KnowCause'] != 0]
ECus = ECus.dropna()
ECus = ECus.reset_index(drop=True)

In [8]:
ECus['DateOutage'] = ECus['DateOutage'].str[:9]
ECus['TimeOutage'] = ECus['TimeOutage'].str[-9:]
ECus['CauseType'] = ECus['CauseType'].str[3:5]
ECus['CausePea'] = ECus['CausePea'].astype(str)
ECus['PEALv'] = ECus['CausePea'].str[4:5]
ECus['Hour'] = ECus['TimeOutage'] .str[1:3]
ECus['Min'] = ECus['TimeOutage'] .str[4:6]
ECus['Region'] = ECus['CausePea'] .str[0:2]
ECus['Day'] = ECus['DateOutage'] .str[0:2]
ECus['Month'] = ECus['DateOutage'] .str[3:5]
ECus['Year'] = '20'+ ECus['DateOutage'] .str[6:8]
ECus['Date'] = ECus['Month']+ '-'+ ECus['Day']+'-'+ ECus['Year']
ECus['Date']= pd.to_datetime(ECus['Date']) 
ECus['Weekday'] = ECus['Date'].dt.dayofweek #starts on Monday, 0 and ends on Sunday,6
ECus['Weekday'] = ECus['Date'].dt.dayofweek
ECus = ECus[['EventID','TotalTime','FeederCode','CauseType','All_Cus','PEALv','Hour','Min','Weekday','Day','Month','Year','Region']]

In [9]:
# event = pd.read_csv('combined_EventCustom_csv.csv')
event = combined_Event_csv
event = combined_Event_csv[['EventID','Weather','ComStatus','ComStart']]

In [10]:
event.loc[:, ('Weather')] = event.loc[:, ('Weather')].str[-2:]
event.loc[:, ('ComStart')]= event.loc[:, ('ComStart')].str[5:6]
event.loc[:, ('ComStatus')]= event.loc[:, ('ComStatus')].str[2:3]

/Users/macbookpro/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [11]:
EquipCode = pd.read_csv('EquipCode.csv')
EquipCode.rename(columns={"EqCode": "ComStart"},inplace = True)
event = event.merge(EquipCode, on='ComStart', how='left')
event['EqType'] = event['EqType'].fillna("EGAT")
event.dropna(subset=['Weather'], inplace = True)

In [12]:
join_event = pd.merge(ECus,  
                     event,  
                     on ='EventID',  
                     how ='left') 
join_event = join_event.dropna()

In [13]:
SumCus = join_event.groupby("EventID", as_index=False)["All_Cus"].sum()
join_event.drop(columns=['All_Cus'],inplace= True)
join_event = pd.merge(join_event,SumCus,on ='EventID',how ='left')
join_event.drop_duplicates(subset =['EventID'],keep = 'first') 

,EventID,TotalTime,FeederCode,CauseType,PEALv,Hour,Min,Weekday,Day,Month,Year,Region,Weather,ComStatus,ComStart,EqType,All_Cus
0,010125610500MCA03H007000,20,MCA03,05,2,05,00,0,01,01,2018,01,02,7,H,F,680.0
1,010125610800CEA06H005000,60,CEA06,05,1,08,00,0,01,01,2018,01,01,7,H,F,366.0
2,010125611124CEA03D001000,19,CEA03,05,5,11,24,0,01,01,2018,01,01,1,D,B,8690.0
4,010125611247LPB04D001000,42,LPB04,01,1,12,47,0,01,01,2018,01,01,1,D,B,25098.0
7,010125611250BGA01H114000,30,BGA01,05,5,12,50,0,01,01,2018,01,01,7,H,F,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497723,150425581045SBY07G002000,60,SBY07,01,4,10,45,2,04,15,2015,12,01,1,G,R,7254.0
497724,260425581500SBY07G002000,90,SBY07,01,4,15,00,6,04,26,2015,12,03,1,G,R,7254.0
497725,290425581700SBY07G002000,117,SBY07,01,4,17,00,2,04,29,2015,12,01,1,G,R,7254.0
497726,300425581650SBY07G005000,60,SBY07,07,4,16,50,3,04,30,2015,12,01,1,G,R,3764.0


In [14]:
ratio = pd.read_csv('All_Ratio.csv') #เปลือยต่อหุ้ม
ratio = ratio[['FEEDERID','Ratio']]
ratio = ratio.rename(columns={"FEEDERID": "FeederCode"})

In [15]:
All_Event = pd.merge(join_event,ratio,on ='FeederCode',how ='left') 
All_Event.loc[All_Event.FeederCode == 'INC01', 'Ratio'] = 1
All_Event['FD'] = All_Event['EventID'].str[12:-7]
All_Event.loc[All_Event['FD'] == All_Event['FeederCode'], 'FeederSameSub'] = 'T'
All_Event['FeederSameSub'] = All_Event['FeederSameSub'].fillna("F")
All_Event['Ratio'] = All_Event['Ratio'].fillna(0)
All_Event['FeederCode'] = All_Event['FeederCode'].str[3:5]

In [16]:
#Feature Engineering - Handling Cyclical Features

convert_dict = {'Weekday': int, 
                'Hour': int,
                'Min': int,
                'Month': int,
               } 
All_Event = All_Event.astype(convert_dict)

import numpy as np

All_Event['hr_sin'] = np.sin(All_Event.Hour*(2.*np.pi/24))
All_Event['hr_cos'] = np.cos(All_Event.Hour*(2.*np.pi/24))
All_Event['min_sin'] = np.sin(All_Event.Min*(2.*np.pi/60))
All_Event['min_cos'] = np.cos(All_Event.Min*(2.*np.pi/60))
All_Event['WDay_sin'] = np.sin(All_Event.Weekday*(2.*np.pi/7))
All_Event['WDay_cos'] = np.cos(All_Event.Weekday*(2.*np.pi/7))
All_Event['mnth_sin'] = np.sin((All_Event.Month-1)*(2.*np.pi/12)) #0-11 0 is Jan
All_Event['mnth_cos'] = np.cos((All_Event.Month-1)*(2.*np.pi/12))

In [17]:
All_Event.drop(['EventID','FD','Hour','Min','Weekday','Month','ComStart'], axis=1, inplace = True )
median = All_Event['TotalTime'].median()
std = All_Event['TotalTime'].std()
outliers = (All_Event['TotalTime'] - median).abs() > std
All_Event['TotalTime'][outliers] = np.nan
All_Event['TotalTime'].fillna(median, inplace=True)

/Users/macbookpro/opt/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
col_cat = ['FeederCode','PEALv','FeederSameSub','ComStatus','Weather','Year','EqType','Region','Day'] #เก็บหัว columns ที่เป็น cat ไว้ใน df
df = All_Event
df[col_cat] = df[col_cat].astype("category")
df_dummy = pd.get_dummies(df[col_cat]) #สร้าง one hot สำหรับตัวแปรที่เป็น cat
df_drop_cat = df.drop(columns = col_cat)
df_transform = pd.concat([df_drop_cat,df_dummy],axis=1) # axis = 1 ต่อแนวยาว ถ้าไม่มี จะต่อ แนวบนลงล่าง
df_transform.shape

(995466, 142)

In [19]:
col_feature = list(df_transform.columns)
df_transform[col_feature] = \
    df_transform[col_feature].astype("float")

In [20]:
df_transform.to_csv('EventForML.csv', encoding='utf-8', quoting=csv.QUOTE_NONE, index=False)

In [21]:
df_transform

,TotalTime,CauseType,All_Cus,Ratio,hr_sin,hr_cos,min_sin,min_cos,WDay_sin,WDay_cos,...,Day_03,Day_04,Day_05,Day_06,Day_07,Day_08,Day_09,Day_10,Day_11,Day_12
0,20.0,5.0,680.0,3.501066,9.659258e-01,0.258819,0.000000,1.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,60.0,5.0,366.0,2.019209,8.660254e-01,-0.500000,0.000000,1.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19.0,5.0,8690.0,1.403414,2.588190e-01,-0.965926,0.587785,-0.809017,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,5.0,8690.0,1.403414,2.588190e-01,-0.965926,0.587785,-0.809017,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,42.0,1.0,25098.0,1.051104,1.224647e-16,-1.000000,-0.978148,0.207912,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995461,36.0,1.0,33920.0,0.254818,5.000000e-01,-0.866025,-0.500000,0.866025,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995462,36.0,1.0,33920.0,0.254818,5.000000e-01,-0.866025,-0.500000,0.866025,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995463,36.0,1.0,33920.0,0.254818,5.000000e-01,-0.866025,-0.500000,0.866025,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995464,36.0,1.0,33920.0,0.254818,5.000000e-01,-0.866025,-0.500000,0.866025,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
list (All_Event.columns)

['TotalTime',
 'FeederCode',
 'CauseType',
 'PEALv',
 'Day',
 'Year',
 'Region',
 'Weather',
 'ComStatus',
 'EqType',
 'All_Cus',
 'Ratio',
 'FeederSameSub',
 'hr_sin',
 'hr_cos',
 'min_sin',
 'min_cos',
 'WDay_sin',
 'WDay_cos',
 'mnth_sin',
 'mnth_cos']

In [24]:
All_Event.head()

,TotalTime,FeederCode,CauseType,PEALv,Day,Year,Region,Weather,ComStatus,EqType,...,Ratio,FeederSameSub,hr_sin,hr_cos,min_sin,min_cos,WDay_sin,WDay_cos,mnth_sin,mnth_cos
0,20.0,03,05,2,01,2018,01,02,7,F,...,3.501066,T,9.659258e-01,0.258819,0.000000,1.000000,0.0,1.0,0.0,1.0
1,60.0,06,05,1,01,2018,01,01,7,F,...,2.019209,T,8.660254e-01,-0.500000,0.000000,1.000000,0.0,1.0,0.0,1.0
2,19.0,03,05,5,01,2018,01,01,1,B,...,1.403414,T,2.588190e-01,-0.965926,0.587785,-0.809017,0.0,1.0,0.0,1.0
3,19.0,03,05,5,01,2018,01,01,1,B,...,1.403414,T,2.588190e-01,-0.965926,0.587785,-0.809017,0.0,1.0,0.0,1.0
4,42.0,04,01,1,01,2018,01,01,1,B,...,1.051104,T,1.224647e-16,-1.000000,-0.978148,0.207912,0.0,1.0,0.0,1.0
